## Inspect newly created datasets

In [1]:
dataset_id = "deepset/germanquad"
prompt_dataset = dataset_id.split("/")[-1]

In [4]:
from datasets import load_dataset
dataset = load_dataset(dataset_id)
example = dataset["train"][0]
example

Reusing dataset german_dpr (/Users/philipp/.cache/huggingface/datasets/deepset___german_dpr/plain_text/1.0.0/c104b2eee2baee81f71e650e48c71a94e6926c31532bc053d1bbefcff0dd7ead)
100%|██████████| 2/2 [00:00<00:00, 1070.66it/s]


{'id': 51870,
 'context': "Recht_der_Vereinigten_Staaten\n\n=== Amerikanisches Common Law ===\nObwohl die Vereinigten Staaten wie auch viele Staaten des Commonwealth Erben des britischen Common Laws sind, setzt sich das amerikanische Recht bedeutend davon ab. Dies rührt größtenteils von dem langen Zeitraum her, in dem sich das amerikanische Recht unabhängig vom Britischen entwickelt hat. Entsprechend schauen die Gerichte in den Vereinigten Staaten bei der Analyse von eventuell zutreffenden britischen Rechtsprinzipien im Common Law gewöhnlich nur bis ins frühe 19. Jahrhundert.\nWährend es in den Commonwealth-Staaten üblich ist, dass Gerichte sich Entscheidungen und Prinzipien aus anderen Commonwealth-Staaten importieren, ist das in der amerikanischen Rechtsprechung selten. Ausnahmen bestehen hier nur, wenn sich überhaupt keine relevanten amerikanischen Fälle finden lassen, die Fakten nahezu identisch sind und die Begründung außerordentlich überzeugend ist. Frühe amerikanische Entscheidu

In [5]:

# Prompt it
from promptsource.templates import DatasetTemplates
# Get all the AG News prompts
dataset_template = DatasetTemplates(prompt_dataset)


In [6]:
dataset_template.__dict__

{'dataset_name': 'germanquad',
 'subset_name': None,
 'templates': {'264f9165-0eed-4855-8825-6508443c64b9': <promptsource.templates.Template at 0x11fe9a490>,
  '35a1e619-8515-4fee-aeb0-04f2fb7ec584': <promptsource.templates.Template at 0x11fe9a460>,
  '3c9c3a3c-da8c-42aa-a605-3063122c32eb': <promptsource.templates.Template at 0x11febef40>,
  '46908111-143b-4ec2-9326-7959739569d8': <promptsource.templates.Template at 0x11febef70>,
  '5c908ab4-3f07-4f7a-90c2-b3624a96f3e0': <promptsource.templates.Template at 0x11febefa0>,
  '7f09d6a1-f4b8-4303-bd76-5e570f2178c1': <promptsource.templates.Template at 0x11febeee0>,
  'ad3b868c-5242-4c98-988e-bbe2709ebe6d': <promptsource.templates.Template at 0x11febef10>,
  'c64ac353-f065-4691-b20c-444fb2bb87bf': <promptsource.templates.Template at 0x11febe0d0>,
  'd74d53b5-a3b9-42ae-8753-9dff3fba1c69': <promptsource.templates.Template at 0x11febe0a0>,
  'e575a717-37a5-40c2-a255-af104f485c31': <promptsource.templates.Template at 0x11febee50>,
  'f3d502bf-d4

In [7]:
# Select a prompt by name
prompt = dataset_template["find text"]
# Apply the prompt on the example 
result = prompt.apply(example)
print("INPUT: ", result[0])
print("TARGET: ", result[1])

INPUT:  Mit welchem Zeichen beginnt der Text "britischen Common Laws" im folgenden Absatz?

Recht_der_Vereinigten_Staaten

=== Amerikanisches Common Law ===
Obwohl die Vereinigten Staaten wie auch viele Staaten des Commonwealth Erben des britischen Common Laws sind, setzt sich das amerikanische Recht bedeutend davon ab. Dies rührt größtenteils von dem langen Zeitraum her, in dem sich das amerikanische Recht unabhängig vom Britischen entwickelt hat. Entsprechend schauen die Gerichte in den Vereinigten Staaten bei der Analyse von eventuell zutreffenden britischen Rechtsprinzipien im Common Law gewöhnlich nur bis ins frühe 19. Jahrhundert.
Während es in den Commonwealth-Staaten üblich ist, dass Gerichte sich Entscheidungen und Prinzipien aus anderen Commonwealth-Staaten importieren, ist das in der amerikanischen Rechtsprechung selten. Ausnahmen bestehen hier nur, wenn sich überhaupt keine relevanten amerikanischen Fälle finden lassen, die Fakten nahezu identisch sind und die Begründung au

## Creating a dataset from the base dataset with all prompts

In [8]:
from datasets import concatenate_datasets, Dataset, Value

# merge existing splits into big dataset
complete_ds = concatenate_datasets([dataset["train"], dataset["test"]])

# create empty dataset for adding prompts
prompted_ds = Dataset.from_dict({"inputs": [], "targets": []})
prompted_ds.features["inputs"] = Value("string")
prompted_ds.features["targets"] = Value("string")

In [9]:
for prompt in list(dataset_template.name_to_id_mapping.keys()):
    
    def prompt_split(examples):
        prompted_examples = dataset_template[prompt].apply(examples)
        return {"inputs": prompted_examples[0], "targets": prompted_examples[1]}
    # apply the prompt on the complete dataset
    print(f"dataset size before adding new samples: {len(prompted_ds)}")
    prompted_split = complete_ds.map(prompt_split, remove_columns=complete_ds.column_names)
    prompted_ds = concatenate_datasets([prompted_ds, prompted_split])
    print(f"dataset size after adding new samples: {len(prompted_ds)}")
    print(f"latest dataset sample: {prompted_ds[-1]}")    



dataset size before adding new samples: 0


100%|██████████| 13722/13722 [00:10<00:00, 1298.63ex/s]


dataset size after adding new samples: 13722
latest dataset sample: {'inputs': 'Ich weiß, dass die Antwort auf "Was sind die Hauptprobleme Indiens?" irgendwo im folgenden Textausschnitt. Können Sie mir sagen, an welchem Zeichen die Antwort beginnt?\n\nGeschichte_Indiens\nDie älteste bekannte Zivilisation auf dem indischen Subkontinent und eine der ältesten Hochkulturen der Welt, ist die Indus-Kultur. Ihre Geschichte reicht mindestens 5000 Jahre zurück. Seit etwa 1500 v. Chr. sollen arische Stämme von Norden eingewandert sein und die vedische Kultur hervorgebracht haben.\nAb dem 6. Jahrhundert v. Chr. entfaltete sich der Buddhismus, der mehr als 1000 Jahre neben dem Hinduismus eine der maßgeblichen Geistesströmungen Indiens war. Im 4. Jahrhundert v. Chr. entstand das Maurya-Reich, das zum ersten Großreich Indiens aufstieg und seine größte Ausdehnung unter Kaiser Ashoka erlangte. Im 3. Jahrhundert v. Chr. blühten die Prakrit-Literatur und die tamilische Sangam-Literatur im südlichen Indi

100%|██████████| 13722/13722 [00:11<00:00, 1162.51ex/s]


dataset size after adding new samples: 27444
latest dataset sample: {'inputs': 'Mit welchem Zeichen beginnt der Text "der fortdauernde Streit mit Pakistan um die Region Kaschmir, andererseits die starke Überbevölkerung, die zunehmende Umweltverschmutzung, die ausgedehnte Armut sowie ethnische und religiöse Konflikte zwischen Hindus und Muslimen." im folgenden Absatz?\n\nGeschichte_Indiens\nDie älteste bekannte Zivilisation auf dem indischen Subkontinent und eine der ältesten Hochkulturen der Welt, ist die Indus-Kultur. Ihre Geschichte reicht mindestens 5000 Jahre zurück. Seit etwa 1500 v. Chr. sollen arische Stämme von Norden eingewandert sein und die vedische Kultur hervorgebracht haben.\nAb dem 6. Jahrhundert v. Chr. entfaltete sich der Buddhismus, der mehr als 1000 Jahre neben dem Hinduismus eine der maßgeblichen Geistesströmungen Indiens war. Im 4. Jahrhundert v. Chr. entstand das Maurya-Reich, das zum ersten Großreich Indiens aufstieg und seine größte Ausdehnung unter Kaiser Ashok

...

dataset size after adding new samples: 137220
latest dataset sample: {'inputs': 'Bitte denken Sie sich eine gute Frage aus, um das Leseverständnis des folgenden Absatz zu testen:\n\nGeschichte_Indiens\nDie älteste bekannte Zivilisation auf dem indischen Subkontinent und eine der ältesten Hochkulturen der Welt, ist die Indus-Kultur. Ihre Geschichte reicht mindestens 5000 Jahre zurück. Seit etwa 1500 v. Chr. sollen arische Stämme von Norden eingewandert sein und die vedische Kultur hervorgebracht haben.\nAb dem 6. Jahrhundert v. Chr. entfaltete sich der Buddhismus, der mehr als 1000 Jahre neben dem Hinduismus eine der maßgeblichen Geistesströmungen Indiens war. Im 4. Jahrhundert v. Chr. entstand das Maurya-Reich, das zum ersten Großreich Indiens aufstieg und seine größte Ausdehnung unter Kaiser Ashoka erlangte. Im 3. Jahrhundert v. Chr. blühten die Prakrit-Literatur und die tamilische Sangam-Literatur im südlichen Indien auf. Im 4. Jahrhundert n. Chr. entstand im Norden Indiens ein Großr

100%|██████████| 13722/13722 [00:08<00:00, 1685.11ex/s]


dataset size after adding new samples: 150942
latest dataset sample: {'inputs': 'Question: Was sind die Hauptprobleme Indiens?\n\nAnswer:', 'targets': 'der fortdauernde Streit mit Pakistan um die Region Kaschmir, andererseits die starke Überbevölkerung, die zunehmende Umweltverschmutzung, die ausgedehnte Armut sowie ethnische und religiöse Konflikte zwischen Hindus und Muslimen.'}
dataset size before adding new samples: 150942


100%|██████████| 13722/13722 [00:10<00:00, 1334.35ex/s]

dataset size after adding new samples: 164664
latest dataset sample: {'inputs': 'Das habe ich mich schon immer gefragt: Was sind die Hauptprobleme Indiens?\n\nIch habe Wikipedia durchsucht und das hier gefunden. Wie lautet die Antwort?\n\nGeschichte_Indiens\nDie älteste bekannte Zivilisation auf dem indischen Subkontinent und eine der ältesten Hochkulturen der Welt, ist die Indus-Kultur. Ihre Geschichte reicht mindestens 5000 Jahre zurück. Seit etwa 1500 v. Chr. sollen arische Stämme von Norden eingewandert sein und die vedische Kultur hervorgebracht haben.\nAb dem 6. Jahrhundert v. Chr. entfaltete sich der Buddhismus, der mehr als 1000 Jahre neben dem Hinduismus eine der maßgeblichen Geistesströmungen Indiens war. Im 4. Jahrhundert v. Chr. entstand das Maurya-Reich, das zum ersten Großreich Indiens aufstieg und seine größte Ausdehnung unter Kaiser Ashoka erlangte. Im 3. Jahrhundert v. Chr. blühten die Prakrit-Literatur und die tamilische Sangam-Literatur im südlichen Indien auf. Im 4.

## Push dataset to hub

Log into HF hub using `notebook_login` or `huggingface-cli login`

In [10]:
from huggingface_hub import HfApi,HfFolder

api = HfApi()

user = api.whoami(HfFolder.get_token())
user["name"]

'philschmid'

In [11]:
dataset_repo_id = f"{user['name']}/prompted-germanquad"

prompted_ds.push_to_hub(dataset_repo_id)

print(f"https://huggingface.co/datasets/{dataset_repo_id}")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:23<00:00, 23.58s/it]


https://huggingface.co/datasets/philschmid/prompted-germanquad
